# Toy example of ImagePixelClassifier and ImagePatchClassifier

This notebook shows the usage of the image classsifiers on a toy example 

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import itertools

Import classifier classes 

In [3]:
from eolearn.ml_tools import ImagePixelClassifier, ImagePatchClassifier

## Create dummy patch classifier

- predict: returns maximum value in an array
- predict_proba: returns maxiumum and minimum value in an array

In [4]:
class DummyPatchClassifier():
    def __init__(self, receptive_field):
        self.receptive_field = receptive_field
    
    def _predict(self, X):
        return np.max(X)
    
    def _predict_proba(self, X):
        return np.max(X), np.min(X)
    
    def predict(self, X):        
        if X.shape[1:3] == self.receptive_field:
            return np.asarray([self._predict(example) for example in X],dtype=np.int)
        else:
            raise ValueError('Dummy Classifier: input of incorrect shape')
        
    def predict_proba(self, X):
        if X.shape[1:3] == self.receptive_field:
            return np.asarray([self._predict_proba(example) for example in X],dtype=np.float)
        else:
            raise ValueError('Dummy Classifier: input of incorrect shape')

## Create dummy data set

50 images of size 10 x 10 x 3 bands

In [5]:
X = np.random.randint(0,100,(50,10,10,3))

In [6]:
X.shape

(50, 10, 10, 3)

In [7]:
X[0,:,:,2]

array([[78, 23, 13, 63, 17, 67, 93, 58, 79, 34],
       [43, 35, 96, 23, 18, 18, 28, 14, 53, 31],
       [98, 94, 87,  2, 73, 62, 31,  4, 86, 12],
       [59, 78, 85, 64, 93, 89, 29, 45, 34, 30],
       [67, 23,  1,  9, 10, 71, 33, 38, 99, 78],
       [37, 99, 75, 16, 86, 20, 48, 55, 39, 33],
       [80, 68, 86, 23,  5,  8, 51,  9, 43, 75],
       [16, 59, 40, 87, 23, 90, 35, 28,  8, 68],
       [81, 46, 72, 14, 49, 89, 56, 67, 18, 94],
       [35, 87, 82, 30,  7, 29, 98, 35, 10, 56]])

## Create `ImagePatchClassifier` and initialise it with `DummyPatchClassifier`

In [8]:
receptive_field = (2,2)
dummy_classifier = DummyPatchClassifier(receptive_field=receptive_field)

In [9]:
patch_classifier = ImagePatchClassifier(dummy_classifier, dummy_classifier.receptive_field)

### `predict` (run classifier over all patches and collect results)

In [10]:
Xprediction = patch_classifier.image_predict(X)

In [11]:
Xprediction.shape

(50, 10, 10)

In [12]:
Xprediction[0]

array([[78., 78., 96., 96., 99., 99., 93., 93., 93., 93.],
       [78., 78., 96., 96., 99., 99., 93., 93., 93., 93.],
       [98., 98., 87., 87., 93., 93., 94., 94., 86., 86.],
       [98., 98., 87., 87., 93., 93., 94., 94., 86., 86.],
       [99., 99., 77., 77., 97., 97., 98., 98., 99., 99.],
       [99., 99., 77., 77., 97., 97., 98., 98., 99., 99.],
       [80., 80., 96., 96., 95., 95., 51., 51., 96., 96.],
       [80., 80., 96., 96., 95., 95., 51., 51., 96., 96.],
       [94., 94., 91., 91., 89., 89., 98., 98., 94., 94.],
       [94., 94., 91., 91., 89., 89., 98., 98., 94., 94.]])

In [13]:
equal = True
for i, j, k in itertools.product(range(0,5), range(0,5), range(0,50)):
    if int(Xprediction[k,2*i,2*j]) != np.max(X[k,2*i:2*i+2,2*j:2*j+2,:]):
        print(Xprediction[k,i,j])
        print(np.max(X[k,2*i:2*i+2,2*j:2*j+2,:]))
        equal = False
        break
    
print(equal)

True


### `predict_proba` (run classifier over all patches and collect results)

In [14]:
Xprobabilities = patch_classifier.image_predict_proba(X)

In [15]:
Xprobabilities.shape

(50, 10, 10, 2)

In [16]:
np.array_equal(Xprediction,Xprobabilities[:,:,:,0])

True

In [17]:
equal = True
for i, j, k in itertools.product(range(0,5), range(0,5), range(0,50)):
    if int(Xprobabilities[k,2*i,2*j,1]) != np.min(X[k,2*i:2*i+2,2*j:2*j+2,:]):
        equal = False
        break
    
print(equal)

True


## Create `DummyPixelClassifier` and `ImagePixelClassifier`

In [18]:
class DummyPixelClassifier():
    def _predict(self, X):
        return np.max(X)
    
    def _predict_proba(self, X):
        return np.max(X), np.min(X)
    
    def predict(self, X):
        return np.asarray([self._predict(example) for example in X],dtype=np.int)
        
    def predict_proba(self, X):
        return np.asarray([self._predict_proba(example) for example in X],dtype=np.float)

In [19]:
dummy_pixel_classifier = DummyPixelClassifier()

In [20]:
pixel_classifier = ImagePixelClassifier(dummy_pixel_classifier)

### `predict` (run classifier over all patches and collect results)

In [21]:
Xprediction = pixel_classifier.image_predict(X)

In [22]:
Xprediction.shape

(50, 10, 10)

In [23]:
np.array_equal(Xprediction, np.max(X,axis=3))

True

### `predict_proba` (run classifier over all patches and collect results)

In [24]:
Xprobabilities = pixel_classifier.image_predict_proba(X)

In [25]:
Xprobabilities.shape

(50, 10, 10, 2)

In [26]:
np.array_equal(Xprobabilities[...,1], np.min(X,axis=3))

True